In [ ]:
!pip install timm

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount= True)
%cd /content/drive/MyDrive/OOD_2/
!ls

In [ ]:
import pandas as pd
import torch
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from CustomImageDataset import CustomImageDataset
from models.SkinLesionLightClassifier import SkinLesionLightClassifier
from models.SkinLesionRuzinClassifier import SkinLesionRuzinClassifier
from train_model import train_model

In [ ]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 32
# percentage of training set to use as validation
valid_size = 0.3
# use a gpu if one is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
path_HAM10000 = ''

In [ ]:
classes = {5: ('nv', ' melanocytic nevi'), 4: ('mel', 'melanoma'), 2 :('bkl', 'benign keratosis-like lesions'), 1:('bcc' , ' basal cell carcinoma'), 6: ('vasc', ' pyogenic granulomas and hemorrhage'), 0: ('akiec', 'Actinic keratoses and intraepithelial carcinomae'),  3: ('df', 'dermatofibroma')}
label_dict = {'akiec' : 0, 'bcc': 1, 'bkl': 2, 'df': 3, 'mel': 4, 'nv':5, 'vasc':6}

In [ ]:
metadata_df = pd.read_csv(path_HAM10000 + 'HAM10000_metadata.csv')

train_unprocessed, validation = train_test_split(metadata_df, test_size=0.3, stratify=metadata_df['dx'],
                                                 random_state=42)
oversample = RandomOverSampler()
y = train_unprocessed['dx'];
x = train_unprocessed.drop(columns=['dx']);
train, y = oversample.fit_resample(x, y)
train['dx'] = y
training_data = CustomImageDataset(train, img_dir='ham10000_images_part_1',
                                   transform=transforms.Compose([
                                       transforms.ConvertImageDtype(torch.float),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.RandomVerticalFlip(),
                                       transforms.RandomAutocontrast()]))
train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True, drop_last= True)
validation_data = CustomImageDataset(validation, img_dir='ham10000_images_part_1')
valid_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=True, drop_last=True)

In [ ]:
# number of epochs to train the model
n_epochs = 25
# initialize tracker for minimum validation loss
early_stopping_window = 4
model = SkinLesionRuzinClassifier(7)
model.to(device)
#model.load_state_dict(torch.load('checkpoints_mobilenet/final_model.pth'))


In [ ]:
model.to(device)
train_model(model, train_loader, valid_loader, n_epochs, early_stopping_window,
            device, last_module_name='classifier',
            save_checkpoints_path='checkpoints_mobilenet/',
            save_model_path='checkpoints_mobilenet/',
            save_figures_path= 'checkpoints_mobilenet/',
            save_checkpoints = True,
            freeze = True)